In [1]:
import os
from transformers import AutoModelForSeq2SeqLM
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load local model
model = (AutoModelForSeq2SeqLM
         .from_pretrained("/kaggle/input/mt5-trained-8-epochs/mt5-summarize-ar/checkpoint-7000/")
         .to(device))

OSError: Can't load the configuration of '/kaggle/input/mt5-trained-8-epochs/mt5-summarize-ar/checkpoint-7000/'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/kaggle/input/mt5-trained-8-epochs/mt5-summarize-ar/checkpoint-7000/' is the correct path to a directory containing a config.json file

In [2]:
import pandas as pd
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory

In [3]:
import pandas as pd
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory

def create_submission(output_file_path : str, submission_dictionary : Dict[int, str], base_keys : List[int]) -> None:
    """Function that validates the submission data types and schema and zip it to be ready from submission
    
    Parameters
    ----------
    output_file_path : str 
        The locaiton and file name you want to save the zip file at, ex : "/home/user/submission_123.zip"
    submission_dictionary : dict[int, str]
        dictionary of int keys (example_id) and string values (summary)
    base_keys: list[int]
        list of keys of the original unlabeled validation set

    
    Returns
    -------
    None
    """
    #assertions
    assert all(isinstance(i, int) for i in submission_dictionary.keys()), "Make sure example_ids elements (key of submission_dictionary) are of type int"
    assert all(isinstance(i, str) for i in submission_dictionary.values()), "Make sure summary elements (value of submission_dictionary) are of type str"
    assert all(isinstance(i, int) for i in base_keys), "Make sure base_keys elements is of type int"
    
    diff_sub = set(submission_dictionary.keys()) - set(base_keys)
    diff_base = set(base_keys) - set(submission_dictionary.keys())
    
    assert len(diff_sub) == 0, f"Keys {diff_sub} is in submission but not in base_keys"
    assert len(diff_base) == 0, f"Keys {diff_base} is in base_keys but not in submission"
    
    #saving
    final_submission = pd.DataFrame(submission_dictionary.items(), columns=['example_id', 'summary'])
    
    if final_submission.example_id.dtype != 'int64' :
        final_submission.example_id = final_submission.example_id.astype(int)
    
    assert len(final_submission[final_submission.summary.isna()]) == 0, f"summaries with the example_id = {final_submission[final_submission.summary.isna()].example_id.values.tolist()} is NaN"
    assert len(final_submission[final_submission.example_id.isna()]) == 0, f"example_ids with the following index = {final_submission[final_submission.example_id.isna()].index.tolist()} is NaN"
    
    with TemporaryDirectory(dir=".") as tmpdirname:
        os.chdir(tmpdirname)
        jsonl_name = "predictions.jsonl"
        final_submission.to_json(jsonl_name, lines=True, orient='records', force_ascii=False)
        with ZipFile(output_file_path, "w") as zip_file:
            zip_file.write(filename = jsonl_name)
            print(f"Submission of {jsonl_name} as .zip saved at {output_file_path}")
        os.chdir("..")

In [4]:
val_data = pd.read_json("/kaggle/input/kaggleinputarabic-summarization-contest/validation_data.jsonl", lines=True)

In [5]:
val_data

,example_id,paragraph
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...
...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...


In [6]:
test1 = val_data["paragraph"][0]
test1

'وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام تعال معا نتناول بيت المقدس بوصفه نموذجا لفتوحات هذه البلاد بشيء من التفصيل .\nفتح بيت المقدس ١٥هـ / ٦٣٦م:\nلحق عمرو بن العاص بالروم، وشدد عليهم الحصار ، ولم يتمكن الروم من فكه، وهرب الأرطبون إلى مصر مركز ثقل الوجود البيزنطي في إفريقيا لصد هجمات المسلمين.\nوأثناء الحصار كانت تجرى محادثات لدخول أهل بيت المقدس في الصلح مع المسلمين، فلم يجد بطريرك بيت المقدس أمامه سوى المصالحة، ولكنه اشترط أن يتم الصلح مع الخليفة عمر بن الخطاب بنفسه وذلك لما للمدينة من قدسية وأهمية دينية كبيرة، فأرسل عمرو بن العاص بهذه الأخبار إلى الخليفة بالمدينة النبوية، وقال له في رسالته: "إني أعالج حربا كؤدًا صدومًا وبلادا ادخرت لك فلك رأيك" فكان أول خروج للخليفة من المدينة بعد موت رسول الله ، هذا الخروج الأجل لكي يفتح بيت المقدس . وكان أول نزول له بمدن الشام بالجابية، والتي وفدت عليه فيها رسل البطريرك طالبين الصلح، فصالحهم عمر صلحا لم تحظ به مدينة أخرى في البلاد المفتوحة.... \nهل تعرف شروط هذا الصلح ؟\nاقرأ النص التالى لتتعرف كيف كان الخليفة عمر سخيا في منحه الامتيازات ل

In [7]:
t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

input_sentences = val_data["paragraph"]
total_sum = []
for i in input_sentences:
    inputs = t5_tokenizer(i, truncation=True, max_length=1024, padding="longest", return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)


    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams=15,
            max_length=150,
            early_stopping=True
        )

    summary = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    total_sum.append(summary)
    
    


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
len(total_sum)

272

In [9]:
# Create a new column 'summary' and populate it with the corresponding summaries from total_sum
val_data['summary'] = [total_sum[example_id] for example_id in val_data['example_id']]

# Print the updated pandas DataFrame
val_data

,example_id,paragraph,summary
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...,أعلنت وزارة الداخلية المصرية عن فتح بيت المقدس...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...,دعا بطريرك بيت المقدس إلى فتح كنيسة القيامة في...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...,فشل المسلمون في فتح شمال إفريقيا بضم النوبة.
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...,فتح مدينة سبيطلة الواقعة في شمال إفريقيا، التي...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...,وصلت حركة الفتوحات الإسلامية إلى عصر بني أمية ...
...,...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...,أعلنت دراسة حديثة عن دور العلماء في الحياة الس...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...,أصبحت صناعة المنسوجات المصرية المرتبة الأولى م...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...,وصلت فرنسا إلى العاصمة الفرنسية باريس بعد خمسة...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...,أصبحت قناة السويس من أهم الطرق الملاحية التي ت...


In [10]:
submission_dictionary = { example_id : summary for _, example_id, summary in val_data[['example_id', 'summary']].itertuples() } 
submission_dictionary

{0: 'أعلنت وزارة الداخلية المصرية عن فتح بيت المقدس في مدينة إيلياء الواقعة شرقي إفريقيا.',
 1: 'دعا بطريرك بيت المقدس إلى فتح كنيسة القيامة في عصر الخلفاء الراشدين.',
 2: 'فشل المسلمون في فتح شمال إفريقيا بضم النوبة.',
 3: 'فتح مدينة سبيطلة الواقعة في شمال إفريقيا، التي يسيطر عليها الجيش البيزنطي على الحدود بين البلدين.',
 4: 'وصلت حركة الفتوحات الإسلامية إلى عصر بني أمية شرقي الصين، وذلك في جولة تاريخية تشهدها البلاد.',
 5: 'أطلقت ولاة بني أمية خريطة الفتوحات الأموية في المشرق الإسلامي.',
 6: 'فشل المسلمين في حصار مدينة القسطنطينية منطقة الصراع بينهم وبين الروم.',
 7: 'أعلنت حركة الفتوح الإسلامية في بلاد الروم فتحها في عهد "عبد الملك بن مروان".',
 8: 'أصبح فتح بلاد المغرب مركزاً لاستكمال الفتوحات في شمال إفريقيا.',
 9: 'أعلنت وزارة الخارجية الإسبانية أن فتح شرق الأندلس قد تم بناء على تخطيط مسبق من قبل المسلمين.',
 10: 'تمكن جيش "شارل مارتل" من فتح النصف الجنوبي لفرنسا إلى الشرق، وذلك بعد أن انتهت معركة بلاد الفرنجة.',
 11: 'أفادت خريطة الفتوحات الإسلامية في عصر الدولة العباسية في بلا

In [11]:
base_keys = val_data.example_id.values.tolist()
len(base_keys)

272

In [12]:
create_submission(output_file_path = "/kaggle/working/new_submission.zip", submission_dictionary = submission_dictionary, base_keys = base_keys)

Submission of predictions.jsonl as .zip saved at /kaggle/working/new_submission.zip
